In [54]:
import base64
import requests

API_URL = "http://0.0.0.0:8000/sam/detect"

data = {
  "image_path": "sample_inp.png",
  "positive_points": [
    [
      614.5455729166666,
      292.8788548519737
    ]
  ],
  "negative_points": [],
  "threshold": 1
}

response = requests.post(API_URL, json=data)



In [55]:
response.content

b'{"name":"temp (18).png","subfolder":"","type":"input","mask_preview":"..."}'

In [43]:
from PIL import Image

response = response.json()
i1 = response["mask_preview"]

i = base64.b64decode(i1)

with open("mask.png", "wb") as f:
    f.write(i)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [32]:
import numpy as np
import cv2
import scipy.sparse
from scipy.sparse.linalg import spsolve

from os import path

def laplacian_matrix(n, m):
    """Generate the Poisson matrix. 

    Refer to: 
    https://en.wikipedia.org/wiki/Discrete_Poisson_equation

    Note: it's the transpose of the wiki's matrix 
    """
    mat_D = scipy.sparse.lil_matrix((m, m))
    mat_D.setdiag(-1, -1)
    mat_D.setdiag(4)
    mat_D.setdiag(-1, 1)
        
    mat_A = scipy.sparse.block_diag([mat_D] * n).tolil()
    
    mat_A.setdiag(-1, 1*m)
    mat_A.setdiag(-1, -1*m)
    
    return mat_A



def poisson_edit(source, target, mask, offset):
    """The poisson blending function. 

    Refer to: 
    Perez et. al., "Poisson Image Editing", 2003.
    """

    # Assume: 
    # target is not smaller than source.
    # shape of mask is same as shape of target.
    y_max, x_max = target.shape[:-1]
    y_min, x_min = 0, 0

    x_range = x_max - x_min
    y_range = y_max - y_min
        
    M = np.float32([[1,0,offset[0]],[0,1,offset[1]]])
    source = cv2.warpAffine(source,M,(x_range,y_range))
        
    mask = mask[y_min:y_max, x_min:x_max]    
    mask[mask != 0] = 1
    #mask = cv2.threshold(mask, 127, 1, cv2.THRESH_BINARY)
    
    mat_A = laplacian_matrix(y_range, x_range)

    # for \Delta g
    laplacian = mat_A.tocsc()

    # set the region outside the mask to identity    
    for y in range(1, y_range - 1):
        for x in range(1, x_range - 1):
            if mask[y, x] == 0:
                k = x + y * x_range
                mat_A[k, k] = 1
                mat_A[k, k + 1] = 0
                mat_A[k, k - 1] = 0
                mat_A[k, k + x_range] = 0
                mat_A[k, k - x_range] = 0

    # corners
    # mask[0, 0]
    # mask[0, y_range-1]
    # mask[x_range-1, 0]
    # mask[x_range-1, y_range-1]

    mat_A = mat_A.tocsc()

    mask_flat = mask.flatten()    
    for channel in range(source.shape[2]):
        source_flat = source[y_min:y_max, x_min:x_max, channel].flatten()
        target_flat = target[y_min:y_max, x_min:x_max, channel].flatten()        

        #concat = source_flat*mask_flat + target_flat*(1-mask_flat)
        
        # inside the mask:
        # \Delta f = div v = \Delta g       
        alpha = 1
        mat_b = laplacian.dot(source_flat)*alpha

        # outside the mask:
        # f = t
        mat_b[mask_flat==0] = target_flat[mask_flat==0]
        
        x = spsolve(mat_A, mat_b)
        #print(x.shape)
        x = x.reshape((y_range, x_range))
        #print(x.shape)
        x[x > 255] = 255
        x[x < 0] = 0
        x = x.astype('uint8')
        #x = cv2.normalize(x, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
        #print(x.shape)

        target[y_min:y_max, x_min:x_max, channel] = x

    return target


# scr_dir = 'figs/example1'
# out_dir = scr_dir
target = cv2.imread("/Users/vishal/Desktop/hack/MedX/backend/Hyundai Creta car.jpeg") 
source = cv2.imread("/Users/vishal/Desktop/hack/MedX/backend/Medx 00016.png")
mask = cv2.imread("/Users/vishal/Desktop/hack/MedX/backend/ComfyUI temp.png", cv2.IMREAD_GRAYSCALE)
offset = (100,100)
result = poisson_edit(source, target, mask, offset)

cv2.imwrite("possion1.png", result)



True

In [30]:
source.shape

(450, 750, 3)

In [25]:
target.shape

(768, 757, 3)

In [26]:
result.shape

(768, 757, 3)